In [2]:
from datetime import datetime
import requests
from bs4 import BeautifulSoup as BS
from konlpy.tag import Mecab
import pandas as pd
from collections import Counter


In [5]:
import grequests

/home/hadoop/miniconda3/lib/python3.10/site-packages/grequests.py:22: MonkeyPatchWarning: Monkey-patching ssl after ssl has already been imported may lead to errors, including RecursionError on Python 3.6. It may also silently lead to incorrect behaviour on Python 3.7. Please monkey-patch earlier. See https://github.com/gevent/gevent/issues/1016. Modules that had direct imports (NOT patched): ['urllib3.util.ssl_ (/home/hadoop/miniconda3/lib/python3.10/site-packages/urllib3/util/ssl_.py)', 'urllib3.util (/home/hadoop/miniconda3/lib/python3.10/site-packages/urllib3/util/__init__.py)']. 
  curious_george.patch_all(thread=False, select=False)


In [6]:
mecab = Mecab()

In [7]:
today = datetime.today().strftime("%Y%m%d")

In [8]:
base_url = "https://news.naver.com/main/list.naver?mode=LSD&mid=shm&sid1=101&date=" + today

In [9]:
def get_news_links(page):
    url = base_url + "&page=" + str(page)
    response = requests.get(url)
    soup = BS(response.text, "html.parser")
    
    links = []
    for a in soup.select("ul.type06_headline li dl dt a"):
        links.append(a["href"])
    for a in soup.select("ul.type06 li dl dt a"):
        links.append(a["href"])
    
    return links

def get_news_content(url):
    response = requests.get(url)
    soup = BS(response.text, "html.parser")
    
    title_tag = soup.select_one("h2.media_end_head_headline")
    content_tag = soup.find('article',{'id':'dic_area'})
    
    if title_tag and content_tag:
        title = title_tag.get_text().strip()
        content = content_tag.get_text().strip()
        return title, content
    else:
        return None, None

In [11]:
# 크롤링 시작
news_links = []
page = 1

while True:
    links = get_news_links(page)
    if not links or any(link in news_links for link in links):
        break
    news_links.extend(links)
    page += 1

RecursionError: maximum recursion depth exceeded

In [ ]:
len(news_links)

In [ ]:
# 뉴스 기사 내용 크롤링
news_contents = []
for link in news_links:
    try:
        title, content = get_news_content(link)
        news_contents.append((title, content))
    except Exception as e:
        print(f"Failed to get content from {link}: {e}")

In [ ]:
#중복값 제거
set(news_contents)

In [ ]:
len(set(news_contents))

In [ ]:

# Create a DataFrame to store titles and nouns
df = pd.DataFrame(columns=["Title", "Nouns"])

# Text analysis using Mecab and store in DataFrame
for i, (title, content) in enumerate(set(news_contents), start=1):
    if content:
        nouns = mecab.nouns(content)
        new_row = pd.DataFrame({"Title": [title], "Nouns": [nouns]})
        df = pd.concat([df, new_row], ignore_index=True)        

# Display the DataFrame
print(df)

In [ ]:
df.to_csv("article_to_Nouns.csv", index=False, encoding='utf-8-sig')

In [ ]:
Counter(df.loc[1,'Nouns'])

In [ ]:
def create_word_count_df(nouns_column):
    # Counter 객체 생성
    counter = Counter()
    
    # Nouns 컬럼의 각 행에 있는 단어들을 업데이트
    for nouns in nouns_column:
        # 각 행의 명사 리스트를 문자열로 읽어들이고, 쉼표와 공백을 제거한 후 리스트로 변환
        # nouns_list = nouns.strip('[]').replace("'", "").split(', ')
        nouns_list = eval(nouns)
        counter.update(nouns_list)
    
    # 중복된 값과 개수를 저장할 리스트
    data = []
    
    # Counter 객체에서 중복된 값과 개수를 추출하여 리스트에 저장
    for item, count in counter.items():
        data.append({'단어': item, '개수': count})
    
    # 데이터프레임 생성
    word_count_df = pd.DataFrame(data)
    
    return word_count_df

In [ ]:
create_word_count_df(df['Nouns'])

In [ ]:
# 기존 방식으로 Nouns 컬럼의 각 행에 있는 단어들의 총합 계산
counter_old = Counter()

for nouns in df['Nouns']:
    # 각 행의 명사 리스트를 문자열로 읽어들이고, 쉼표와 공백을 제거한 후 리스트로 변환
    # nouns_list = nouns.strip('[]').replace("'", "").split(', ')
    counter_old.update(nouns)

In [ ]:
counter_old

In [14]:
import aiohttp
import asyncio
import pandas as pd
from bs4 import BeautifulSoup
from konlpy.tag import Mecab
from datetime import datetime


In [39]:
import aiohttp
import asyncio
import pandas as pd
from bs4 import BeautifulSoup
from konlpy.tag import Mecab
from datetime import datetime
import nest_asyncio

# nest_asyncio 적용
nest_asyncio.apply()

mecab = Mecab()

async def fetch(session, url):
    try:
        async with session.get(url, timeout=aiohttp.ClientTimeout(total=600)) as response:
            return await response.text()
    except asyncio.TimeoutError:
        print(f"Timeout error for URL: {url}")
        return None

async def get_news_links(session, base_url, page):
    url = f"{base_url}&page={page}"
    response_text = await fetch(session, url)
    if response_text is None:
        return []
    soup = BeautifulSoup(response_text, 'html.parser')
    
    links = []
    for a in soup.select("ul.type06_headline li dl dt a"):
        links.append(a["href"])
    for a in soup.select("ul.type06 li dl dt a"):
        links.append(a["href"])
    
    return links

async def get_news_content(session, url):
    response_text = await fetch(session, url)
    if response_text is None:
        return None, None
    soup = BeautifulSoup(response_text, 'html.parser')
    
    title_tag = soup.select_one("h2.media_end_head_headline")
    content_tag = soup.find('article', {'id': 'dic_area'})
    
    if title_tag and content_tag:
        title = title_tag.get_text().strip()
        content = content_tag.get_text().strip()
        return title, content
    return None, None

async def main():
    today = datetime.today().strftime("%Y%m%d")
    base_url = "https://news.naver.com/main/list.naver?mode=LSD&mid=shm&sid1=101&date=" + datetime.today().strftime("%Y%m%d")
    async with aiohttp.ClientSession() as session:
        news_links = []
        page = 1
        
        while True:
            links = await get_news_links(session, base_url, page)
            # if not links or any(link in news_links for link in links):
            #     break
            if not links or set(links) & set(news_links):
                break            
            news_links.extend(links)
            page += 1
        
        news_links = set(news_links)
        tasks = [get_news_content(session, link) for link in news_links]
        news_contents = await asyncio.gather(*tasks)
        
        # None 값을 제거
        news_contents = [content for content in news_contents if content[0] is not None]

        # 명사 추출
        nouns_data = []
        for title, content in news_contents:
            # nouns = mecab.nouns(content)
            nouns_data.append({'Title': title, 'content': content})
        
        df = pd.DataFrame(nouns_data)
        df.to_csv(f"{today}.csv", index=False, encoding='utf-8-sig')

# Jupyter Notebook 환경에서 실행
await main()


In [40]:
pd.read_csv(f"{today}.csv",encoding='utf-8')

,Title,content
0,"한전KPS, 대국민 혁신 아이디어 공모…14일까지 접수",(나주=연합뉴스) 송형일 기자 = 발·송전 설비 정비 전문회사인 한전KPS는 오는 ...
1,"""우체국도 카카오페이 됩니다""... 현금·카드 없어도 간편 결제","우정사업본부, 1일부터 우체국 창구서 우편, 소포 요금 등 카카오페이로 결제 가능\..."
2,"iM뱅크, 여름 휴가 시즌 카드 해외결제 캐시백 이벤트",iM뱅크 BC카드 해외 결제 시최대 5만원 캐시백 \n\n\n\n[파이낸셜뉴스]iM...
3,"현대차, 인도네시아 교두보로 '아세안 공략' 속도",현대차 인도네시아 생산법인(HMMI) 전경현대차가 아세안 지역 공략에 나섰다. 각국...
4,초코파이 2개·조각수박 ‘소포장 시대’,소용량 매출 기존대비 최대 3배1~2인 가구 증가·고물가에 급증‘필요한 양만 사자’...
...,...,...
4364,국제안전보건전시회 개막,(고양=뉴스1) 장수영 기자 = 1일 오후 경기 고양시 킨텍스에서 열린 2024 국...
4365,"中알리·테무 이용자 수 두 달 만에 반등…""대규모 세일 반짝효과""",6월 알리·테무 월간 활성 이용자 수(MAU) 증가세1위는 쿠팡 3129만명…전달 ...
4366,"[시사이슈 찬반토론] 툭하면 '헬조선', 한국은 양극화 심화 사회인가",한국 사회를 비하하는 말 가운데 ‘헬조선’이 있다. 선거철이 되면 이런 구호를 외치...
4367,7000억 매몰비용 물거품…백지화된 'CJ라이브시티'의 꿈,"경기도, 30일 CJ측에 협약 해제 통보2017년 매입한 A·C부지도 반환해야100..."
